# 空污物聯網數據分析與資料視覺化(1) - 
# ***<span style='color:red'>AIoT and Python Plolty Go</span>*** 繪圖模組應用
> [民生公共物聯網資料服務平台](https://ci.taiwan.gov.tw/dsp/dataset_air_epa_micro.aspx)網頁中有提供全國環保署佈建的智慧城鄉空品微型感測器之位置及感測資料, 本專案利用資料下載另行整理後再開始以Python進行數據分析及資料視覺化, 從大數據的觀點來觀察一些感測器想要告訴我們的事, 主要目標是練習數據分析手法, 但是重點不在環保相關議題的討論   
> 將會用到python 的一些特色功能及資料處理模組 **pandas dataframe(df)** 的功能包括:  
> - 讀取檔案並建立df(read_csv)
> - 選取(iloc/loc, column name)
> - 篩選(filter/mask)
> - 取用與刪除
> - 迴圈處理(enumerate)
> - list comprehensive
> - geopandas與GeoJSON的轉換與應用
 
> 資料視覺化的部分, 我們將應用Python基於javascrip D3.js包裝的的互動式繪圖模組 **plotly express(px)** 及 **graphic object(go)** 實作下列功能,包括:
> - 線型圖multiple lines in a chart with range selector, range slider and across-type marker line
> - 散布圖time-based scatter chart
> - 圖資上的散布圖及周界圖location scatter and choropleth map on open street map

| Robert Lin <nobodybutyou.lin@gmail.com> 

## 目標與Overview
依據實際生活周遭環境大數據, 應用Python數據分析與繪圖工具, 發掘大數據資料科學的花園美景  
本專案採用的環保署智慧城鄉空品微型感測器數據資料, 篩選某特定工業區內***200顆感測器***, 每個感測器每隔一分鐘傳送一筆數據, 除了溫濕度外還有PM2.5資料, 選定特定一天大約近20小時的數據資料收集, 近**23萬筆空品數據**  
在此我們不討論環保界討論的數據品質,與環保署大型空品測站的相關性, 溫濕度的影響等, 純粹從大數據的角度做數據發掘與工具淬煉
另外, 我們也會以政府開放資訊平台中的縣市鄉鎮區邊界資料集及鄉鎮市區人口密度資料集, 透過Python Plotly go在空間地圖上展現資料視覺化效果

## 主要輸入檔內容格式 
在民生公共物聯網資料平台上所提供的空品微型感測器數據量方面, 全國一天約有***500MB***以上, 包含感測器位置座標, 我們的輸入檔案經過調整與處理合併後, 欄位如下圖: 
<img src="inputFileColumnName.jpg">

## 執行前確認檔案目錄結構
程式有既定讀取和寫出檔案, 可按照程式中指定路徑放置或修改程式, 主要有下列四個目錄: 
```
data/dataset7441/TOWN_MOI_1100415.shp  鄉鎮市區周界資料集
data/dataset8410/opendata109N010-peopleD.csv  鄉鎮市區人口密度資料集 
data/iot/20211010chungli-pmvoc.csv  空品微型感測器數據資料
geojson/  
log/  
token/  
```
我是在macbook上執行, 如果換成在windows上執行, 路徑的表達方式不同, 有需要修改程式碼, 將"/"換成"`\`"反斜線  
繪圖時如果地圖是選用open street map就不需要mapbox的access token  
執行完成後有***幾個互動式的圖形輸出html***會放在此程式下的log目錄之下,  
```
log/area1zleft.html 所有感測器的小時均值z分數散布圖  
log/dev1areachartleft.html 整體區域的5分鐘平均值及5分鐘標準差趨勢線圖  
log/area1locmap.html 感測器所在的位置分佈地圖, 及鄉鎮市區分界與人口密度
```

## 程式與說明

順便練習一下使用等直線圖(choropleth map)來繪製縣市鄉鎮區邊界, 並以人口密度來標示顏色, 邊界資料可在政府公開資訊平台上下載, [dataset是7441](https://data.gov.tw/dataset/7441) , 下載資料來看是ArcGIS SHP(shape)檔案格式, 解壓縮後可以看到有許多檔案, 主要要有.shp .shx .dbf檔案, 相關檔案用途就看網路上的參考資料了  
繪圖模組的樣式挑選可以到[Python Graph Libraries官網](https://plotly.com/python/) , 最重要的line chart模組相關示範案例也可查詢[python go line char應用示範](https://plotly.com/python/line-charts/)  
首先, 我們將會用到的模組import進來, 如果你的電腦中沒有plotly模組則需要另外透過pip或conda安裝  
python另外還有一支互動式繪圖模組叫Dash, 但看起來plotly就夠用了, plotly express的延伸graphic object(go)的設計邏輯接近, 較易上手, 就先從這裡下手, 我們把一些數據分析用的模組另外放在airdev.py中

In [1]:
# https://data.gov.tw/dataset/7441
# 直轄市、縣市界線(TWD97經緯度)SHP格式

# Python Graph libraries
# https://plotly.com/python/

# 各種go line chart應用示範
# https://plotly.com/python/line-charts/
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import datetime

import pandas as pd
import json
import numpy as np

import airdev

## 提升效率的dataframe取用
***注意:*** Pandas dataframe在操作上應避免chain index以面造成解譯程式時發生太多SettingWithCopyWarning,要盡量多用loc+mask 詳細說明可參考[View or Copy](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy) 


## 內建print列印行數不夠用
當dataframe資料數很長時, 內建的print功能只能看到前後幾筆資料, 可以透過 ***pd.set_options('max_rows', None)*** 及 ***pd.set_options(‘max_cloumns', None)*** 來調整為無限制行列數

## 鄉鎮市區人口密度統計
在政府資訊公開平台上有鄉鎮市人口密度統計資料[dataset 8410](https://data.gov.tw/dataset/8441), 檔案內的欄位名稱如下圖:
<img src="dataset8410peopleD.jpg"> 

因為縣市與鄉鎮區是寫在同一欄, 所以我們訂一個函數叫dfColumnSplit來將一欄分出兩欄資料
下列程式碼的函數中第一行被remark掉, 雖然功能和第二行一樣目的, 但是這種就是前面所提到的 ***chain indexing*** , 在執行python interpreter編譯時會出現warning, 盡量養成習慣避免使用, 雖然在網路上很多範例也都這樣寫

In [2]:
def dfColumnSplit(df, i, delimiterChar):
    # pos=df['site_id'][i].find(delimiterChar)
    pos =df.loc[i, 'site_id'].find(delimiterChar)
    if pos>=0:
        df.loc[i, 'site_id'] = df.loc[i,'site_id'][:pos+1] + ' ' + df.loc[i,'site_id'][pos+1:]
        afStr=df.loc[i,'site_id'].split(' ')
        #print(afStr)
        df.loc[i,'countyName'] = afStr[0]
        df.loc[i,'townName']=afStr[1]

pd.set_option('max_rows', None)
pd.set_option('max_columns', None)

#mapbox_access_token = open("token/.taiwanMapboxAccessToken").read()

## 讀入主要感測數據檔
原始資料讀入後存在dataframe ***dfAir*** , 我們先透過drop_duplicates取出唯一的裝置名稱並刪掉不需要的header column(含有EUI字串),  
***dfNewAir*** 將存放所有各裝置的名稱及座標等主要資料,  
***dfDevs*** 欄位較少且經緯度經過精密度處理, 只留小數點後4位, 可精密至 ***11公尺***

In [3]:
#---------Process ChungLi industry area sensor dev data
dfAir=pd.read_csv('data/iot/20211010chungli-pmvoc.csv', header=0, parse_dates =["時間"],encoding='utf-8')

# Checking dfDevs will be used for device map drawing
#取出dataframe中的uniqu entry且不需要原來df的index (ignore_index)
dfNewAir = dfAir.drop_duplicates(subset=['裝置名稱','lat', 'lon'], ignore_index=True)
dfNewAir = dfNewAir.loc[dfNewAir.loc[:,'EUI']!='EUI',:]

#再取出unique entry中的三個欄位就好
dfDevs = dfNewAir.loc[:, ['EUI', '裝置名稱','lat', 'lon']]
dfDevs.loc[:, 'lat'] = dfDevs.loc[:,'lat'].astype(float)

# Use .loc to do copy is more efficient
dfDevs.loc[:,'lon'] = dfDevs.loc[:,'lon'].astype(float)
dfDevs.loc[:,'lat']=round(dfDevs.loc[:,'lat'], 4)
dfDevs.loc[:,'lon']=round(dfDevs.loc[:,'lon'], 4)

# 資料清洗
dfAir 裡面還有不需要的header column(含EUI字串), 我們透過df[filer].index的方式來取出需要被丟掉列的index, 然後透過dfAir.drop的方法將這些列去除, 且直接在dfAir這個object上執行生效(inplace=True), 
前面在讀此csv時的parse_date似乎無效, 在這裡明確指定date time的format再透過 ***pd.to_datetime*** 方法, 將字串轉為python內的datetime格式

In [4]:
#根據dataframe 某一欄的值來刪除列
#https://www.delftstack.com/zh-tw/howto/python-pandas/drop-row-pandas/
maskindex=dfAir[dfAir['EUI']=='EUI'].index
dfAir.drop(index=maskindex, inplace=True)

# have to parse according to datetime format
dfAir.loc[:,'時間'] = pd.to_datetime(dfAir.loc[:,'時間'], format="%Y/%m/%d %H:%M")

# 數據分析
我們希望透過短時間(5分鐘)的感測器數值分析, 看是否可發現 ***相同空間中的異常表現數據*** 來輔助判斷可能的異常污染事件 
另外我們也希望透過較長時間(1小時)的感測數據, 來看感測數據在 ***群體中的測值落點分佈*** 看是否可發現長時間異常表現的感測器
我們先準備 ***5分鐘的區域平均值和區域標準差*** 所需要的資料  
airdev模組中我們定義了calDevAvg來計算各個sensor device的5分鐘平均值, 在airdev模組中透過python的resample就可依據時間規格在dfAir重新取樣及計算, 所以這個函數的第三個參數就是取樣的時間規格, 第二個參數則是可指定要做重新取樣的欄位名稱
各個計算完成5分鐘平均值後回傳會存在df5TDevAvg  
calAreaStdev是用來依據輸入參數中的dataframe(df5TDevAvg)在指定欄位('PM2.5')的值來計算整體區域的標準差, 因為輸入的dataframe是各感測器5分鐘的平均值, 求出來的區域標準差就是5分鐘的區域標準差,   
calAreaAvg是用來計算整體區域指定間隔時間(5T)的區域平均值, 這裡時間間隔使用5分鐘(5T)

In [5]:
df5TDevAvg = airdev.calDevAvg(dfAir, 'PM2.5', '5T') #calculate 5 min avg

dfStdev = airdev.calAreaStdev(df5TDevAvg, 'PM2.5')

df5TAreaAvg = airdev.calAreaAvg(dfAir, 'PM2.5', '5T') #Calculate 1 min dfAvgGrp

dfLineMerge = dfStdev.merge(df5TAreaAvg, how='inner', \
                         left_on='時間', right_on='時間', left_index=True)
dfLineMerge.loc[:,'stdev']=round(dfLineMerge.loc[:,'stdev'], 1)
dfLineMerge.loc[:,'avg']=round(dfLineMerge.loc[:,'avg'], 1)
#print(dfLineMerge)

In func calDevAvg, timeSpecStr is .... 5T
In func calAreaStdev......
In func calAreaAvg, timeSpecStr is..... 5T
In func calDevAvg, timeSpecStr is .... 5T


## 計算小時均值Z分數
***z=(小時均值-區域小時平均值)/區域小時標準差***,  
z分數的值在統計上有其一般常態分配狀況下的所在位置百分比的意義,  
代表***測值在群體中與區域平均值有幾個標準差的距離*** 但先決條件是要跟區域小時平均值比  
所以我們要求出各個感測器的小時均值, 也要計算區域內整體感測器的小時平均值以及標準差  
***注意:*** 這裡有個python dataframe依據mask篩選條件後要插入有特定值的欄位之用法  
***插入值的list內元素數量必須與mask過濾後的符合條件列數*** 相同, 所以我們用list comprehensive來造這個list   
為了能夠利用python的recursive特性, 最後我們會在df1HDevAvg中各個device的1小時均值列, 插入areaAvg及areaStdev欄位, 如此即可一行程式計算各device的小時均值z分數, 從這裡也可看到python利用空間來換取的 ***平行處理*** 潛力  
```
df1HDevAvg.loc[:,'z']=(df1HDevAvg.loc[:,'PM2.5'] - df1HDevAvg.loc[:, 'areaAvg'])/df1HDevAvg.loc[:,'areaStdev']   
```

In [6]:
# z=(x-mean)/stdev
df1HDevAvg = airdev.calDevAvg(dfAir, 'PM2.5', '60min')

df1HAreaAvg = airdev.calAreaAvg(df1HDevAvg, 'PM2.5', '60min')
df1HAreaStdev = airdev.calAreaStdev(df1HDevAvg, 'PM2.5')


# fill the value of areaAvg and areaStdev of df1HDevAvg from  1HAreaAvg.avg and 1HAreadStdev.stdev
uniqueLst = df1HDevAvg.loc[:,'裝置名稱'].unique()
df1HDevAvg.insert(df1HDevAvg.shape[1], column='areaAvg', value=None)
df1HDevAvg.insert(df1HDevAvg.shape[1], column='areaStdev', value=None)

for timei, timeItem in enumerate(df1HAreaAvg.loc[:,'時間']):
        
        valuea= df1HAreaAvg.loc[df1HAreaAvg['時間']==timeItem, 'avg']
        values= df1HAreaStdev.loc[df1HAreaStdev['時間']==timeItem, 'stdev']
        mask = (df1HDevAvg.loc[:,'時間']==timeItem)
        
        #create array with fixed size and the same values
        valueaT=[valuea for i in range(sum(mask))]
        valuesT=[values for i in range(sum(mask))]
        
        df1HDevAvg.loc[mask, 'areaAvg']=valueaT
        df1HDevAvg.loc[mask, 'areaStdev']=valuesT


df1HDevAvg.insert(df1HDevAvg.shape[1], column='z', value=None)
# calculate z value
df1HDevAvg.loc[:,'z']=(df1HDevAvg.loc[:,'PM2.5'] - df1HDevAvg.loc[:, 'areaAvg'])/df1HDevAvg.loc[:,'areaStdev']

In func calDevAvg, timeSpecStr is .... 60min
In func calAreaAvg, timeSpecStr is..... 60min
In func calDevAvg, timeSpecStr is .... 60min
In func calAreaStdev......


## 整理數據
計算結果有很多小數點, 我們透過 ***round*** 方法, 限定各欄位的小數點位數, 避免有些欄位沒有處理乾淨, 用 ***try...except*** 來捕捉意外狀況, 讓程式不會因資料錯誤而無法繼續執行

In [ ]:
for i in df1HDevAvg.index:
    try:
        df1HDevAvg.loc[i,'z']=round(df1HDevAvg.loc[i, 'z'], 3)
        df1HDevAvg.loc[i,'PM2.5']=round(df1HDevAvg.loc[i,'PM2.5'], 3)
        df1HDevAvg.loc[i,'areaAvg']=round(df1HDevAvg.loc[i,'areaAvg'], 3)
        df1HDevAvg.loc[i,'areaStdev']=round(df1HDevAvg.loc[i,'areaStdev'], 3)
    except Exception as e:
        print('round off error for item index i...', i, e)
        print(df1HDevAvg.loc[i, :])

繪圖之前有個函數蠻好用的, 當呼叫python go或plotly express(px)模組時, 可指定在圖形顯示時的title標題文, 但是如果標題文太長, 可以用下列這個函數mutipleStringLines做切割, 使得在呼叫px.scatter時title指定的標題參數內容, 可以在圖形顯示時以多行方式呈現

In [ ]:
#https://stackoverflow.com/questions/35185143/how-to-create-new-line-in-plot-ly-js-title
# Deploy multiple lines title
def multipleStringLines(title):
    if (len(title) > 20) : # check if greater than threshold!
            y_axis = title.split(' ')    #break into words
            interval = int(len(title.split(' '))/ 3 )    #3-lines
            return (' '.join(y_axis[0:interval]))+ '<br>' + (' '.join(y_axis[interval:interval*2]))+ \
              '<br>'+(' '.join(y_axis[interval*2: len(y_axis)]))

    return title


## 繪製各device的小時均值z分數散布圖
先整理繪圖函數所需要的資料, 在計算應收資料總數時, 取出數據起始與結束時間後, 透過時間差的 ***total_seconds*** 可以換算出時間差的總秒數  
這張圖是呼叫python的 ***plotly express(px)繪圖模組***  
最後的互動式繪圖也可以 ***write_html*** 方式輸出, 直接以browser點選輸出的html檔, 仍可以滑鼠互動式的查詢圖形內的資料, ***hover_data*** 參數可用來指定當滑鼠移到數據點位上時要顯示哪些資料  
 

## 小時均值z分數散布圖呈現的數據意涵初探
簡要說明z分數散布圖所呈現的意涵, 先看看[wiki z-分數](https://zh.wikipedia.org/wiki/%E6%A8%99%E6%BA%96%E5%88%86%E6%95%B8)的介紹, 我們引用內容中的圖示如下來做說明, 前面提到從z分數的公式上, 可以解讀為測值與平均值之間有多少個標準差的距離, wiki的圖示中可以看出, 在常態分配條件下, ***正負兩個標準差之內的測值應該佔95.44%***  
應用上有一個重要的觀察基礎是, ***在一般正常情況下, 同一時間內在一定空間範圍內的感測器測值應該是彼此相當的***, 因此這裡的應用中, 在公式中的平均值, 我們是取相同空間內(工業區)的所有感測器的小時平均值來計算區域小時平均值, 應此從python畫出來的圖來看, 大部分的測值小時均值z分數也都在正負2之間, 超過這個範圍的值就特別需要注意, 長期偏低太多的, 有可能感測器壞掉或阻塞, 固定時段偏高太多的, 就要關心一下周邊是否有固定時間出現的高值污染源, 這些都要更多的環境觀察與佐證才可釐清原因, 但是***透過大數據分析的結果, 已經可以讓我們從上千上萬個感測器中, 縮小範圍至需要關注及深入調查的感測器***     
舉例來說, 中午和晚上用餐時段z分數固定偏高到5以上, 肯定是怪怪的, 環保署空品感測器的架設位置通常都在3公尺高度的路燈桿上, 測值除了可能受周邊環境污染源影響而升高外(例如露天露燒, 工廠細微粒狀物污染排放, 高污染車輛經過), 也有可能受附近的降暑水霧機, 初一十五附近店家燒金紙, 剛好有人在附近抽菸等因素影響而升高  
有趣的是未來可否從這些大量數據中, 透過各種可能污染(火災, 露燃, 道路鋪設工程等)的數據特徵推斷, 出現的時間及週期頻率, 安裝地點的環境照片或google街景圖等各種特徵值, 自動分辨推薦可能的污染行為  
另外一個觀察是如果把z分數計算中的區域平均值換成感測器自己一天的平均值, 然後求各時段對此本身日平均值的z分數, 那代表什麼意義? 它是代表各時段的測值相對於本身一日均值的高低表現, 看到的是一天之內這個感測器的高值時段和低值時段, 但是這個z在整體區域內就不太有跨感測器的高低代表意義, 而且一天內同一感測器的PM2.5高低值變化本來就不會相當,想清楚公式應用的內涵才能找到實體世界的對應意涵  
<img src="wikiz-score.jpg">

In [ ]:
print('Calculating data and drawing graph z-score scatter chart ...')
#dataframe可能做過drop後沒有reset index所以避免直接拿len當index抓最後一個
#若要一致, 要設inplace=True將object內的index重新排序
dfAir.reset_index(inplace=True)

timediff = dfAir.loc[dfAir.shape[0]-1, '時間'] - dfAir.loc[0, '時間']
perDevRec = (timediff.total_seconds() / 60)+1

totalExpDevRec = perDevRec * len(dfNewAir)
totalRecRate = round((dfAir.shape[0]/totalExpDevRec)*100, 2)

titlex1='中壢工業區2021-10-10時均值z分數散布圖 [z=(小時均值-區域小時均值)/區域小時標準差] 註：標記時間t的數據是統計t(含)到t+1之間 '
titlex=titlex1+"微感器數量:"+str(len(dfNewAir))+" 感測數據總筆數:"+str(dfAir.shape[0])+"("+str(totalRecRate)+"%)"
titler=multipleStringLines(titlex)

figz = px.scatter(df1HDevAvg, x="時間", y="z", \
                 title=titler,
                 hover_data=['裝置名稱', 'PM2.5','z','areaAvg', 'areaStdev'])
figz.show()
figz.write_html('log/area1zleft.html')

## 繪製整體區域5分鐘標準差及區域5分鐘平均值 趨勢線圖
這個圖則是採用另一個python繪圖模組Graphic Object(go), 輸入的***dfLineMerge***資料中取出兩欄資料各畫一條line, 其中標準差曲線因為***mode=lines+markers***所以除了連線外, 還會標示數據點  
另外我們也透過***update_layout***在圖形下方增加***range selector及slider***, 可用滑鼠動態選擇顯示範圍

## 區域5分鐘標準差曲線呈現的數據意涵初探
因為我們計算的區域5分鐘標準差, 是計算整體工業區內所有感測器的5分鐘均值的平均值所呈現的標準差, 平常如果空品環境固定, 這個標準差的值不會很大, 也就是相同空間中的感測器測值在彼此間的變異度不會很大, 既便是境外空污來臨後整體測值偏高, 也都是所有感測器一起高, 變異度也不會很大, 除非當下5分鐘內發現了離群偏高值, 就有可能是現地出現污染源, 但嚴重的污染源例如火災,工廠爆炸, 看不見的工廠細微污染粒狀物排放擴散等, 群體感測器的測值在時間序列上會出現隨氣象條件而傳遞與擴散的現象  
為什麼我們以5分鐘的精細度來看空品變異程度, 因為有些污染擴散可能只有10分鐘～20分鐘的高值擴散效應, ***5分鐘的精細度可以及時發現污染,也可避免因時間過長的小時均值計算而讓變異程度不明顯***, 那另一個思考是會不會看到一些不是我們關心的現地污染現象? 這是有可能的, 所以還要透過這個高值現象有沒有擴散影響鄰近的感測器, 不過這就是判斷上的trade-off, 因為有些小範圍的露天燃燒事件, 並未能在群體感測器上看到擴散效果, ***但是這些事件的收集累積後, 你將會發現在規律性週期發生的事件點附近還真的有事***

In [ ]:
print('Drawing stdev and avg line chart ...")
fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=dfLineMerge['時間'], y=dfLineMerge['stdev'], name='5分鐘區域標準差',
                         mode='lines+markers', line=dict(color='firebrick', width=3)))

fig.add_trace(go.Scatter(x=dfLineMerge['時間'], y=dfLineMerge['avg'], name = '區域5分鐘平均值ug/m3',
                         line=dict(color='royalblue', width=3)))

fig.update_layout(title='2021-10-10 中壢工業區 微型空品感測器 數據線圖',
                   xaxis_title='時間',
                   yaxis_title='ug/m3')

# Add range slider
# https://plotly.com/python/reference/layout/xaxis/#layout-xaxis-rangeselector
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=6,
                     label="6小時內",
                     step="hour",
                     stepmode="todate"),
                dict(count=12,
                     label="12小時內",
                     step="hour",
                     stepmode="todate"),
                dict(count=1,
                     label="1日內",
                     step="day",
                     stepmode="todate"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        showspikes=True,
        spikemode="across+marker",
        spikesnap="hovered data",
        type="date",
    )
)


fig.show()
fig.write_html('log/dev1areachartleft.html')


## 繪製鄉鎮區周界及感測器位置圖


這一段我們另外嘗試使用plotly go模組來繪圖, 這裡的 ***go.Choroplethmapbox*** 需要使用GeoJSON格式當作幾何資料輸入, 而政府資訊平台的縣市鄉鎮區邊界資料格式是 ***.shp*** , 所以這裡有需要一個處理手法是將.shp讀入成 ***geopandas dataframe(gdf)*** , 再透過 ***gdf.to_file*** 方法, 將檔案轉匯輸出成 ***GeoJSON格式*** , 之後呼叫go繪圖模組前 ***(go.Figure(go.Choroplethmapbox())*** 再將此檔案讀入成GeoJSON格式資料(twTownJSON)當作go模組繪圖時輸入的資料參數  
python 裡的地理資料處理結構是採用類似pandas的geopandas, 一開始從.shp檔讀入成geopandas dataframe ***gdf_TwTown*** 後續還會放一些人口密度資料, 而go.Choroplethmapbox所需要的幾何圖形座標資料則要求放在GeoJSON格式的輸入參數中(geojson= ***twTownJSON中*** ), 這兩者之間 ***(geopandas與GeoJSON)*** 繪圖模組go要怎麼知道如何關聯在一起呢? 那就要透過go.Choroplethmapbox函數的featureid參數 ***(featureidkey="properties.TOWNCODE")*** 
政府資訊平台的鄉鎮區邊界圖的多邊形幾何座標相當精細, 為了提升處理速度及滿足實際精確度需求即可, 這裡有個手法是將GeoJSON資料裡幾何座標轉成小數點後4位(精細度可到11公尺)的做法  
```
twTownJSON["features"][i]['geometry']['coordinates'][j] = \
                np.round(np.array(twTownJSON["features"][i]['geometry']['coordinates'][j]),4)  
```
下圖是GeoJSON檔案內容的範例  
<img src="geojsonExample.jpg">  

我們希望鄉鎮市區邊界內區域的顏色可用該區的人口密度等級來區別, 因此我們需要根據人口密度資料來做等級分別, 使得繪圖時不同區域的顏色以此人口密度級數欄位來對應顏色 ***(z=gdf_TwTown["PopDCat"])***


In [ ]:
print('Preparing data and drawing choropleth graph.....')
#scattermapbox using go
#https://plotly.com/python/scattermapbox/#set-marker-symbols
site_lat = dfDevs.lat
site_lon = dfDevs.lon
locations_name = dfDevs['裝置名稱']



#Further Study
#GIS空間資料分析,  空間自相關
#--------------Process town region border data
gdf_TwTown=gpd.read_file('data/dataset7441/TOWN_MOI_1100415.shp',encoding='utf-8')

#Study for GeoJSON
#export to GeoJSON format
gdf_TwTown.to_file("geojson/TOWN_MOI_1100415.json", driver='GeoJSON')

#------------Process people density data
# https://data.gov.tw/dataset/8410
dfPD=pd.read_csv("data/dataset8410/opendata109N010-peopleD.csv", encoding='utf-8')
#print(dfPD)
# 以多個定介符分割字串
# https://www.delftstack.com/zh-tw/howto/python/how-to-split-string-with-multiple-delimiters-in-python/
dfPD.insert(dfPD.shape[1], column='countyName', value=None)
dfPD.insert(dfPD.shape[1], column='townName', value=None)
#遍歷 dataframe 的列
# https://www.delftstack.com/zh-tw/howto/python-pandas/how-to-iterate-through-rows-of-a-dataframe-in-pandas/
for i in dfPD.index:
    if  (i > 0) & (type(dfPD.loc[i, 'site_id']) == str):
        dfColumnSplit(dfPD,i,'縣')
        dfColumnSplit(dfPD,i,'市')
indexLst = dfPD[ dfPD['statistic_yyy'].isnull() ].index

# Delete these row indexes from dataFrame
dfPD.drop(indexLst , inplace=True)
#print(dfPD)

#-------------Merge town border and people density data
gdf_TwTown = gdf_TwTown.merge(dfPD, how='inner',\
                         left_on=['COUNTYNAME','TOWNNAME'], right_on=['countyName','townName'], left_index=True)
#print(gdf_TwTown)

gdf_TwTown.insert(dfPD.shape[1], column='PopDCat', value=None)
gdf_TwTown['PopDCat'] = round(gdf_TwTown['population_density'].astype(float), 1)

# Classify PopDCat
for i in gdf_TwTown.index:
    if 0<=gdf_TwTown.loc[i,'PopDCat']<=100:
        gdf_TwTown.loc[i,'PopDCat']=0
    elif 100<=gdf_TwTown.loc[i,'PopDCat']<=200:
        gdf_TwTown.loc[i,'PopDCat']=1
    elif 200<=gdf_TwTown.loc[i,'PopDCat']<=500:
        gdf_TwTown.loc[i,'PopDCat']=2
    elif 500<=gdf_TwTown.loc[i,'PopDCat']<=1000:
        gdf_TwTown.loc[i,'PopDCat']=3
    elif 1000<=gdf_TwTown.loc[i,'PopDCat']<=1500:
        gdf_TwTown.loc[i,'PopDCat']=4
    elif 1500<=gdf_TwTown['PopDCat'][i]<=2000:
        gdf_TwTown.loc[i,'PopDCat']=5
    elif 2000<=gdf_TwTown.loc[i,'PopDCat']<=2500:
        gdf_TwTown.loc[i,'PopDCat']=6
    elif 2500<=gdf_TwTown.loc[i,'PopDCat']<=3500:
        gdf_TwTown.loc[i,'PopDCat']=7
    elif 3500<=gdf_TwTown.loc[i,'PopDCat']<=5000:
        gdf_TwTown.loc[i,'PopDCat']=8
    elif 5000<=gdf_TwTown.loc[i,'PopDCat']<=10000:
        gdf_TwTown.loc[i,'PopDCat']=9
    elif 10000<=gdf_TwTown.loc[i,'PopDCat']<=15000:
        gdf_TwTown.loc[i,'PopDCat']=10
    elif 15000<=gdf_TwTown['PopDCat'][i]<=20000:
        gdf_TwTown.loc[i,'PopDCat']=11
    elif 20000<=gdf_TwTown.loc[i,'PopDCat']<=30000:
        gdf_TwTown.loc[i,'PopDCat']=12
    else:
        gdf_TwTown.loc[i,'PopDCat']=13

maxPopD = gdf_TwTown.loc[:,'PopDCat'].max()
#print("maxPopDCat=", maxPopD)


#---------Process GeoJSON file
with open("geojson/TOWN_MOI_1100415.json") as f:
  twTownJSON = json.load(f)
print('GeoJSON file')
print(twTownJSON["features"][0]['properties'])
#print(twTownJSON["features"][0]['geometry'])

#Round off the locations to 4 decimal places (about 11m accuracy)
for i in range(0, len(twTownJSON["features"])):
    for j in range(0,len(twTownJSON["features"][i]['geometry']['coordinates'])):
        try:
            #print(twTownJSON["features"][i]['properties'][j]))
            twTownJSON["features"][i]['geometry']['coordinates'][j] = \
                np.round(np.array(twTownJSON["features"][i]['geometry']['coordinates'][j]),4)
        except Excepion as e:
            print('round off error',e,i,j)
print('0,0...',twTownJSON["features"][0]['geometry']['coordinates'][0][0])


#--------------------Combine choropleth and scatter
#https://community.plotly.com/t/how-can-i-combine-choropleth-and-scatter-layer-in-a-plotly-map/29842/6

#choropleth map using go
#https://plotly.com/python/mapbox-county-choropleth/#choropleth-map-using-plotlygraphobjects-and-carto-base-map-no-token-needed
#https://plotly.com/python/mapbox-county-choropleth/#choropleth-map-using-plotlygraphobjects-and-carto-base-map-no-token-needed
#choropleth_mapbox using go
# API Ref  https://plotly.com/python-api-reference/generated/plotly.graph_objects.Choroplethmapbox.html
fig = go.Figure(go.Choroplethmapbox(
        geojson=twTownJSON, locations=gdf_TwTown["TOWNCODE"], z=gdf_TwTown["PopDCat"],
                featureidkey="properties.TOWNCODE",
                marker=go.choroplethmapbox.Marker(
                    opacity=0.3
                ),
                name="鄉鎮區代碼", text = gdf_TwTown["TOWNNAME"],
                                    colorscale="Viridis_r", zmin=0, zmax=13, below='')
                                    #marker_opacity=0.5,r marker_line_width=0)
)


fig.add_scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers+text',
        marker=go.scattermapbox.Marker(
            size=12,
            color='rgb(0, 0, 255)',
            opacity=0.5
        ),
        text=locations_name,
        showlegend=False,
        name="微型感測器",
        hoverinfo=['text','lat', 'lon']
    )

# build-in mapbox layer
# https://plotly.com/python/mapbox-layers/#openstreetmap-tiles-no-token-needed
fig.update_layout(
    title='桃園市中壢工業區微型空品感測器分佈圖', title_x=0.5,
    autosize=True,
    hovermode='closest',
    showlegend=True,
    mapbox=dict(
        #accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat= 24.9799,
            lon=121.2445
        ),
        pitch=0,
        zoom=13,
        style='light'
    ),
    mapbox_style="open-street-map"
    #mapbox_style="satellite-streets"
)
fig.write_html('log/area1locmap.html')
fig.show()


## 總結
因為專案目標在實用python功能, 就來摘要一下一些好用的特色及需要的習慣  
- 個人還是比較喜歡plotly express及go, 細部的繪圖元素控制還有很多可控參數可以設定
- 從傳統程式語言(C, Java)出發的人, 看到Python dataframe的取用,設定,刪除,一定有很多容易混淆的用法, 但是如果從***loc[index, column]*** 這樣的行列式方式來取用, 就會比較習慣, python的建議用法也是如此, 另一方面在處理速度和語言文法上的混淆也會改善
- 許多網路上的範例在呼叫方法或函數時省略了參數名稱, 加進來可以增加程式可讀性
- pd.read_csv讀進來的資料是字串型態, 後續要做數值運算, 須先透過 ***astype()*** 轉換型態, 例如:
```
      dfDevs.loc[:, 'lat'] = dfDevs.loc[:,'lat'].astype(float)
```
- 有些函數或方法有 ***inplace參數***, 如果沒有指定為True, 將只會改變object view, 不會改變objec本身, 例如reset_index(inplace=True)
- 有些函數或方法或屬性的保留字容易混淆, 另外提出來做區別, ***columns和column***, 例如:
```
      dfPD.insert(dfPD.shape[1], column='countyName', value=None)
      pandas.DataFrame(data=None, index=None, columns=None, dtype=None, copy=None)
      DataFrame.insert(loc, column, value, allow_duplicates=False)
```
- 過濾篩選條件一大串, 和欄位取用及計算寫在一行程式的可讀性不好, 另外設定mask變數為設定的篩選條件, 再使用 ***df.loc[mask,column]***, 這就容易讀了, 例如以下:
```
      maskindex=dfAir[dfAir['EUI']=='EUI'].index
      dfAir.drop(index=maskindex, inplace=True) #過濾掉符合過濾條件的row index並且修改在原object上
```
- 時間差是timedelta object, 要取出時間差, 可以透過 ***total_seconds()*** 轉換, 用法如下:
```
      perDevRec = (timediff.total_seconds() / 60)+1
```
- python的獨特用法 ***list comprehensive*** , 可以透過以下這個例子來了解
```
      valueaT=[valuea for i in range(sum(mask))] #造一個大小為sum(mask)的list且元素值都是valuea
```
- 透過 ***join*** 分隔字元 ***將list中的所有元素串接*** 起來
     * ' '.join(y_axis[0:interval]) 使用' '為分隔字元, 將y_axis內從0到interval(不含)之間的元素串接成一個新字串
- 透過 ***重新取樣方法resample*** 將指定欄位(如下面的colSrcStr欄)基於datetime欄(on)的時間搓依據指定取樣時間長短規格('60T'表示60分鐘, 或 用'60min')重新取樣計算, 指定將結果計在(label)起始點(left)或結束點(right), 並可指定結束點或時間點的資料是否包含(close參數)在計算內(apply), 此方法如果df的index所以不連續, 則須適用reset_index(), 這個方法也是 ***平行處理*** 可以發揮的地方, 程式中用到的方法如下: 
```
      dfAvg=dfGrpRow.resample(timeSpecStr, on='時間', closed='left', label='left')[colSrcStr].apply(np.mean).reset_index() #resampled values are stored in dfAvg
```
- 計算平均值或標準差如果欄位有 ***Nan空值時的處理方法***, 寫法如下: 
```
      aAvg=np.nanmean(alist)
      astdev=np.nanstd(alist, ddof=len(alist)/2) #ddof: delta degree of freedom, 允許可排除的Ｎan值的數量
```
- plotly figure可透過 ***write_html*** 方法將繪圖結果輸出成html檔, 前端程式的互動式效果仍可保持, 輸出方式如下:
```
      fig.wirte_html('log/are1z.html')  
```

在數據分析的思路邏輯上, 我們應用5分鐘的區域數據統計來嘗試發現***突發的異常事件***, 應用每個感測器的小時均值在區域中的相對位置的z分數, 來嘗試發現區域中***常態性的高值感測器與高值區間***

## 其他參考資料
外部連結的參考資料可在文章中提及的地方直接點選, 程式碼中也納入相關的Delftstack或plotly官網或plotly community文件的參考連結  
另外撰寫本篇文章所用的Markdown語言示範案例有很多, 可以請google依據你的問題搜尋參考答案或可直接參考[卡斯柏的部落格](https://wcc723.github.io/development/2019/11/23/ten-mins-learn-markdown/)